In [4]:
from sympy import *

In [9]:
dx, dy = symbols('dx, dy')
PfC, PfS, PfW, PfE, PfN = symbols('PfC, PfS, PfW, PfE, PfN')
k_ef_W, k_ef_E, k_ef_S, k_ef_N = symbols('k_ef_W, k_ef_E, k_ef_S, k_ef_N')
PtC, eta_phi = symbols('PtC, e_phi')
BCPfW, BCPfE, BCPfS, BCPfN = symbols('BCPfW, BCPfE, BCPfS, BCPfN')

qxW = (1-BCPfW) * -k_ef_W*(PfC-PfW)/dx + BCPfW * 0
qxE = (1-BCPfE) * -k_ef_E*(PfE-PfC)/dx + BCPfE * 0 
qyS = (1-BCPfS) * -k_ef_S*(PfC-PfS)/dy + BCPfS * 0
qyN = (1-BCPfN) * -k_ef_N*(PfN-PfC)/dy + BCPfN * 0
fp  = (qxE-qxW)/dx + (qyN-qyS)/dy - (PtC-PfC)/eta_phi

print(julia_code(fp.diff(PfS)))
print(julia_code(fp.diff(PfW)))
print(julia_code(fp.diff(PfC)))
print(julia_code(fp.diff(PfE)))
print(julia_code(fp.diff(PfN)))

-k_ef_S .* (1 - BCPfS) ./ dy .^ 2
-k_ef_W .* (1 - BCPfW) ./ dx .^ 2
1 ./ e_phi + (k_ef_N .* (1 - BCPfN) ./ dy + k_ef_S .* (1 - BCPfS) ./ dy) ./ dy + (k_ef_E .* (1 - BCPfE) ./ dx + k_ef_W .* (1 - BCPfW) ./ dx) ./ dx
-k_ef_E .* (1 - BCPfE) ./ dx .^ 2
-k_ef_N .* (1 - BCPfN) ./ dy .^ 2


In [12]:
VyS, VyN, VxW, VxE = symbols('VyS, VyN, VxW, VxE')
fp = (VxE-VxW)/dx + (VyN-VyS)/dy + (PtC-PfC)/eta_phi 
print(julia_code(fp.diff(VxW)))
print(julia_code(fp.diff(VxE)))
print(julia_code(fp.diff(VyS)))
print(julia_code(fp.diff(VyN)))
print(julia_code(fp.diff(PtC)))

-1 ./ dx
1 ./ dx
-1 ./ dy
1 ./ dy
1 ./ e_phi


In [29]:
# Stokes x
VxC, VxW, VxE, VxS, VxN = symbols('VxC, VxW, VxE, VxS, VxN')
VySW, VySE, VyNW, VyNE  = symbols('VySW, VySE, VyNW, VyNE')
PtW, PtE = symbols('PtW, PtE')
eW, eE, eS, eN = symbols('eW, eE, eS, eN')
NeuS, DirS, NeuN, DirN = symbols('NeuS, DirS, NeuN, DirN')
#---------------------------------------#
divVW = (VxC-VxW)/dx + (VyNW-VySW)/dy
ExxW  = (VxC-VxW)/dx - Rational(1,3)*divVW
TxxW  = 2*eW*ExxW
#---------------------------------------#
divVE = (VxE-VxC)/dx + (VyNE-VySE)/dy
ExxE  = (VxE-VxC)/dx - Rational(1,3)*divVE
TxxE  = 2*eE*ExxE
#---------------------------------------#
dVxdy = (1 - NeuS - DirS) * (VxC-VxS)/dy + NeuS * 0 + DirS * (2*VxC)
ExyS  = Rational(1,2) * ( dVxdy + (VySE-VySW)/dx )
TxyS  = 2*eS*ExyS
#---------------------------------------#
dVxdy = (1 - NeuN - DirN) * (VxN-VxC)/dy + NeuN * 0 + DirN * (2*VxC)
ExyN  = Rational(1,2) * ( dVxdy + (VyNE-VyNW)/dx )
TxyN  = 2*eN*ExyN
#---------------------------------------#
f = (TxxE-TxxW)/dx + (TxyN-TxyS)/dy - (PtE-PtW)/dx
f *= -1
#---------------------------------------#
print(julia_code(f.diff(VxS)))
print(julia_code(f.diff(VxW)))
print(julia_code(f.diff(VxC).simplify()))
print(julia_code(f.diff(VxE)))
print(julia_code(f.diff(VxN)))
print(julia_code(f.diff(VySW)))
print(julia_code(f.diff(VySE)))
print(julia_code(f.diff(VyNW)))
print(julia_code(f.diff(VyNE)))
print(julia_code(f.diff(PtW)))
print(julia_code(f.diff(PtE)))

-eS .* (-DirS - NeuS + 1) ./ dy .^ 2
-4 // 3 * eW ./ dx .^ 2
(3 * dx .^ 2 .* (-eN .* (2 * DirN .* dy + DirN + NeuN - 1) + eS .* (2 * DirS .* dy - DirS - NeuS + 1)) + 4 * dy .^ 2 .* (eE + eW)) ./ (3 * dx .^ 2 .* dy .^ 2)
-4 // 3 * eE ./ dx .^ 2
-eN .* (-DirN - NeuN + 1) ./ dy .^ 2
-eS ./ (dx .* dy) + (2 // 3) * eW ./ (dx .* dy)
-2 // 3 * eE ./ (dx .* dy) + eS ./ (dx .* dy)
eN ./ (dx .* dy) - 2 // 3 * eW ./ (dx .* dy)
(2 // 3) * eE ./ (dx .* dy) - eN ./ (dx .* dy)
-1 ./ dx
1 ./ dx
